Title: Joining Tables
Slug: sql/joining-tables
Category: SQL
Tags: SELECT, FROM, WHERE, LIMIT, JOIN, INNER JOIN, OUTER JOIN, LEFT JOIN, RIGHT JOIN, CROSS JOIN, UNION, UNION ALL
Date: 2017-08-09
Modified: 2017-08-09

#### Recommended reading
I recommend reading the [Coding Horror's explanation of SQL joins](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/) for more detail. Here are a few examples for reference.

#### Load ipython-sql extension

In [2]:
# The 2 lines below prevent an error message from being displayed when we run %load_ext sql
import warnings
warnings.filterwarnings('ignore')

%load_ext sql
%config SqlMagic.feedback = False

#### Connect to the database

In [3]:
%%sql

postgresql://localhost/dvdrental

'Connected: None@dvdrental'

#### Joining tables
When joining tables, it's handy to give them a shorter alias: this makes it easier to refer to them later. This is important as some tables may have common column names and it's important to differentiate between them.

Note that this method works in PostgreSQL, but you may need to add `AS` after the table name in other SQL implementations.

In [4]:
%%sql

SELECT
    c.customer_id
    , c.first_name
    , c.last_name
    , a.address
    , a.district
    , a.city_id
FROM
    customer c
    -- To be explicit, INNER JOIN can also be used here
    JOIN address a on a.address_id = c.address_id
LIMIT
    5

customer_id,first_name,last_name,address,district,city_id
1,Mary,Smith,1913 Hanoi Way,Nagasaki,463
2,Patricia,Johnson,1121 Loja Avenue,California,449
3,Linda,Williams,692 Joliet Street,Attika,38
4,Barbara,Jones,1566 Inegl Manor,Mandalay,349
5,Elizabeth,Brown,53 Idfu Parkway,Nantou,361


#### Different types of join
In addition to `INNER JOIN`, you can also use `OUTER JOIN`, `LEFT JOIN`, `RIGHT JOIN`, and `CROSS JOIN`. Since the syntax is essentially the same, I'll explain how they work.

#### `INNER JOIN`
Find records in table 1 that have a match in table 2, ignoring records that don't appear in both tables. This is how the example above works.

#### `OUTER JOIN`
Preserve all records from both tables. If a record from table 1 has a match in table 2, join them together. If a record from table 1 doesn't have a match in table 2, return the `NULL` values for the columns from table 2 (and vice versa).

#### `LEFT JOIN` & `RIGHT JOIN`
In a `LEFT JOIN`, all data from the left table is preserved, with matching data from the right table join where it exists. In a `RIGHT JOIN`, the data from the right table is preserved instead.

The 'leftmost' table is the first table specified in your query. For example, in the query above `customer` is the left table and `address` is the right table.

#### `CROSS JOIN`
Match all possible combinations of two tables. As you might expect, this can lead to very large tables and is probably the least common type of join. To deomnstrate, let's `CROSS JOIN` a small sub-selection on the `film` and `language` tables:

In [5]:
%%sql

SELECT
    f.film_id
    , f.title
FROM
    film f
WHERE
    f.film_id < 4

film_id,title
1,Academy Dinosaur
2,Ace Goldfinger
3,Adaptation Holes


In [7]:
%%sql

SELECT
    l.language_id
    , l.name
FROM
    language l
WHERE
    l.language_id < 4

language_id,name
1,English
2,Italian
3,Japanese


In [9]:
%%sql

SELECT
    f.film_id
    , f.title
    , l.language_id
    , l.name
FROM
    film f
    CROSS JOIN language l
WHERE
    f.film_id < 4
    AND l.language_id < 4

film_id,title,language_id,name
1,Academy Dinosaur,1,English
1,Academy Dinosaur,2,Italian
1,Academy Dinosaur,3,Japanese
2,Ace Goldfinger,1,English
2,Ace Goldfinger,2,Italian
2,Ace Goldfinger,3,Japanese
3,Adaptation Holes,1,English
3,Adaptation Holes,2,Italian
3,Adaptation Holes,3,Japanese


#### `UNION`
Combine the results of two or more `SELECT` statements into a single table. Returned results are stacked one above the other, not side-by-side as with the `JOIN`s listed above:

In [13]:
%%sql

SELECT
    a.actor_id
    , a.first_name
    , a.last_name
FROM
    actor a
WHERE
    a.actor_id < 4

actor_id,first_name,last_name
2,Nick,Wahlberg
3,Linda,Williams
3,Ed,Chase
1,Mary,Smith
1,Penelope,Guiness
2,Patricia,Johnson


In [11]:
%%sql

SELECT
    c.customer_id
    , c.first_name
    , c.last_name
FROM
    customer c
WHERE
    c.customer_id < 4

customer_id,first_name,last_name
1,Mary,Smith
2,Patricia,Johnson
3,Linda,Williams


In [14]:
%%sql

SELECT
    a.actor_id
    , a.first_name
    , a.last_name
FROM
    actor a
WHERE
    a.actor_id < 4
    
UNION

SELECT
    c.customer_id
    , c.first_name
    , c.last_name
FROM
    customer c
WHERE
    c.customer_id < 4

actor_id,first_name,last_name
2,Nick,Wahlberg
3,Linda,Williams
3,Ed,Chase
1,Mary,Smith
1,Penelope,Guiness
2,Patricia,Johnson


If there were any duplicate rows in these results, `UNION` would have removed them. To preserve all rows including duplicates, use `UNION ALL`.